<h1>Requests products-by-category</h1>

In [31]:
import requests
import json

import requests

url = "https://real-time-amazon-data.p.rapidapi.com/products-by-category"

querystring = {"category_id":"2478868012","page":"2","country":"US","sort_by":"RELEVANCE","product_condition":"ALL","is_prime":"false","deals_and_discounts":"NONE"}

headers = {
	"x-rapidapi-key": "7c104397e4msh6cb34ab38468193p1f14f0jsn0827f2a925e4",
	"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
}
request = requests.get(url, headers=headers, params=querystring)
response = request.json()

with open('dados_!.json', 'w') as f:
    json.dump(response, f, indent=4)


In [33]:
import pandas as pd
import json

# Caminho para o arquivo
with open('dados.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Acessa apenas a lista de produtos
produtos = data['data']['products']

# Transforma em DataFrame
df = pd.DataFrame(produtos)
len(df)


48

-------------

<h1>Requests products-detail</h1>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, regexp_replace, when


spark = SparkSession\
        .builder\
        .master("local[*]")\
        .appName("amazon_pipeline")\
        .getOrCreate()
        
spark

your 131072x1 screen size is bogus. expect trouble
25/04/20 21:45:30 WARN Utils: Your hostname, PcArthur resolves to a loopback address: 127.0.1.1; using 172.23.223.44 instead (on interface eth0)
25/04/20 21:45:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/20 21:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
import json
path = "/home/arthur/Projetos/amazon_pipeline/data/produtos_1_US.json"
with open(path, "r") as f:
    raw_data = json.load(f)

products = raw_data['data']['products']

jsonlines_path = "/home/arthur/Projetos/amazon_pipeline/data/produtos_1_US_lines.json"

with open(jsonlines_path, 'w') as f:
    for product in products:
        f.write(json.dumps(product) + '\n')
    

In [43]:
df_br = spark.read.json("/home/arthur/Projetos/amazon_pipeline/data/produtos_1_BR_lines.json")
df_br = df_br.drop("is_best_seller", "is_amazon_choice", "is_prime", "product_badge", "climate_pledge_friendly")
df_br.show(5)

+----------+-----------+-----------+--------+--------------------+--------------+--------------+---------------------------+------------------+-------------------+----------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+
|      asin|book_format|coupon_text|currency|            delivery|has_variations|product_byline|product_minimum_offer_price|product_num_offers|product_num_ratings|product_original_price|       product_photo|product_price|product_star_rating|       product_title|         product_url|        sales_volume|
+----------+-----------+-----------+--------+--------------------+--------------+--------------+---------------------------+------------------+-------------------+----------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+
|B09T55NFSR|       NULL|       NULL|     BRL|Receba até Amanhã...|          true|    

In [58]:
df_br = df_br.withColumn("product_price", when(col("product_price").isNull(), 0)
                .otherwise(regexp_replace(col("product_price"), f"[R$ ,]", "").cast("float")))\
        .withColumn("product_original_price", when(col("product_original_price").isNull(), 0)
                .otherwise(regexp_replace(col("product_original_price"), f"[R$ ,]", "").cast("float")))\
        .withColumn("product_minimum_offer_price", when(col("product_minimum_offer_price").isNull(),0)
                .otherwise(regexp_replace(col("product_minimum_offer_price"), f"[$ ,]", "").cast("float")))\
        .withColumn("product_star_rating", col("product_star_rating").cast("float"))\
        .withColumn("product_minimum_offers", col("product_star_rating").cast("float"))\
        .withColumn("product_num_offers", col("product_num_offers").cast("float"))\
        .withColumn("product_num_ratings", col("product_num_ratings").cast("float"))\
        .withColumn("country", lit("BR"))\
        .drop("is_best_seller", "is_amazon_choice", "is_prime", "product_badge", "climate_pledge_friendly", "product_availability", "delivery", "has_variations", "coupon_text", "book_format", "product_byline")\
        .filter(col("product_title").startswith("Apple iPhone"))


In [59]:
df_br.printSchema()

root
 |-- asin: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_minimum_offer_price: float (nullable = true)
 |-- product_num_offers: float (nullable = true)
 |-- product_num_ratings: float (nullable = true)
 |-- product_original_price: float (nullable = true)
 |-- product_photo: string (nullable = true)
 |-- product_price: float (nullable = true)
 |-- product_star_rating: float (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- sales_volume: string (nullable = true)
 |-- product_minimum_offers: float (nullable = true)
 |-- country: string (nullable = false)



In [60]:
df = spark.read.json("/home/arthur/Projetos/amazon_pipeline/data/produtos_1_US_lines.json")

df = df.withColumn("product_price", when(col("product_price").isNull(), 0)
                            .otherwise(regexp_replace(col("product_price"), f"[$ ,]", "").cast("float")))\
                .withColumn("product_original_price", when(col("product_original_price").isNull(), 0)
                            .otherwise(regexp_replace(col("product_original_price"), f"[$ ,]", "").cast("float")))\
                .withColumn("product_minimum_offer_price", when(col("product_minimum_offer_price").isNull(), 0)
                            .otherwise(regexp_replace(col("product_minimum_offer_price"), f"[$ ,]", "").cast("float")))\
                .withColumn("product_star_rating", col("product_star_rating").cast("float"))\
                .withColumn("product_minimum_offers", col("product_star_rating").cast("float"))\
                .withColumn("product_num_offers", col("product_num_offers").cast("float"))\
                .withColumn("product_num_ratings", col("product_num_ratings").cast("float"))\
                .withColumn("country", lit(f"US"))\
                .drop("is_best_seller", "is_amazon_choice", "is_prime", "product_badge", "climate_pledge_friendly", "product_availability", "delivery", "has_variations")

In [61]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_minimum_offer_price: float (nullable = true)
 |-- product_num_offers: float (nullable = true)
 |-- product_num_ratings: float (nullable = true)
 |-- product_original_price: float (nullable = true)
 |-- product_photo: string (nullable = true)
 |-- product_price: float (nullable = true)
 |-- product_star_rating: float (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_url: string (nullable = true)
 |-- sales_volume: string (nullable = true)
 |-- product_minimum_offers: float (nullable = true)
 |-- country: string (nullable = false)



In [62]:
df_total = df.unionByName(df_br)


In [63]:
df_total.show()

+----------+--------+---------------------------+------------------+-------------------+----------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+----------------------+-------+
|      asin|currency|product_minimum_offer_price|product_num_offers|product_num_ratings|product_original_price|       product_photo|product_price|product_star_rating|       product_title|         product_url|        sales_volume|product_minimum_offers|country|
+----------+--------+---------------------------+------------------+-------------------+----------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+----------------------+-------+
|B0BN733951|     USD|                     300.86|              49.0|             4612.0|                 389.0|https://m.media-a...|        357.8|                4.1|Apple iPhone 14, ...|https://www.amazo...|2K+ bough